In [1]:
import tensorflow as tf

## Create tf dataset from a list

In [2]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

### Iterate through tf dataset

In [3]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


### Iterate through tf dataset using numpy elements

In [4]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


### Iterate through 1st n elements in tf dataset

In [5]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


## Filter sales no. that are < 0

In [6]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


## Convert sales numbers from USA dollars to Indian Rupees (INR) Assuming 1-> 72 conversion rate 

In [7]:
tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


## Shuffle

In [8]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1584
1512
2232
2448
2232
2304


## Batching

In [9]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1512 2232]
[2304 1584]
[2232 2448]


# Perform all of the above operations in one shot

In [10]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1512 1584]
[2232 2304]
[2448 2232]


# Images

In [13]:
images_ds = tf.data.Dataset.list_files('datasets/flower_photos/*/*', shuffle=False)

In [14]:
image_count = len(images_ds)
image_count

3670

In [16]:
type(images_ds)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [17]:
for file in images_ds.take(3):
    print(file.numpy())

b'datasets\\flower_photos\\daisy\\100080576_f52e8ee070_n.jpg'
b'datasets\\flower_photos\\daisy\\10140303196_b88d3d6cec.jpg'
b'datasets\\flower_photos\\daisy\\10172379554_b296050f82_n.jpg'


In [18]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'datasets\\flower_photos\\daisy\\14507818175_05219b051c_m.jpg'
b'datasets\\flower_photos\\daisy\\14330343061_99478302d4_m.jpg'
b'datasets\\flower_photos\\daisy\\18442919723_d1251d3e14_n.jpg'


In [19]:
class_names = ["daisy","dandelion", "roses", "sunflowers", "tulips"]

In [20]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [21]:
len(train_ds)

2936

In [22]:
len(test_ds)

734

In [23]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [25]:
get_label("datasets\\flower_photos\\roses\\12240303_80d87f77a3_n.jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'roses'>

In [26]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [27]:
img, label = process_image("datasets\\flower_photos\\roses\\12240303_80d87f77a3_n.jpg")
img.numpy()[:2]

array([[[236.3125  , 246.59375 , 229.98438 ],
        [234.39062 , 250.53125 , 222.3125  ],
        [237.64062 , 249.14062 , 234.39062 ],
        [241.51562 , 252.54688 , 231.5625  ],
        [239.45312 , 252.6875  , 227.76562 ],
        [243.51562 , 252.5625  , 239.53125 ],
        [248.07812 , 252.28125 , 230.79688 ],
        [239.04688 , 251.10938 , 235.125   ],
        [209.0625  , 208.54688 , 218.75    ],
        [176.26562 , 148.35938 , 189.54688 ],
        [153.75    , 118.25    , 164.48438 ],
        [143.5     , 100.140625, 144.90625 ],
        [149.6875  ,  90.265625, 133.375   ],
        [149.64062 ,  85.203125, 128.875   ],
        [152.48438 ,  90.46875 , 122.8125  ],
        [153.875   ,  88.984375, 112.515625],
        [153.10938 ,  90.171875, 114.3125  ],
        [150.0625  ,  87.890625, 117.640625],
        [144.59375 ,  85.859375, 132.42188 ],
        [155.39062 , 115.625   , 171.89062 ],
        [170.96875 , 145.57812 , 205.73438 ],
        [200.21875 , 207.29688 , 2

In [28]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [29]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[ 8.277344  12.277344  12.183594 ]
  [ 9.277344  10.277344  12.996094 ]
  [ 5.2036743  9.203674  10.203674 ]
  ...
  [77.08539   74.45703   50.06195  ]
  [73.3241    76.540955  49.458923 ]
  [69.90625   72.90625   43.90625  ]]

 [[ 3.6640625 10.167969   8.90625  ]
  [ 5.8320312 10.107605  11.71875  ]
  [ 4.611023   8.94696    9.778992 ]
  ...
  [74.14905   74.99054   47.256165 ]
  [76.88672   75.88672   45.826355 ]
  [68.73828   71.73828   42.73828  ]]

 [[ 3.6132812 13.561951  11.1328125]
  [ 4.7734375 12.253906  12.71875  ]
  [ 5.6132812 11.613281  11.613281 ]
  ...
  [68.87225   69.4035    45.161316 ]
  [77.35321   77.13898   46.746094 ]
  [67.29297   70.29297   41.29297  ]]

 ...

 [[18.722107  19.335388  21.891174 ]
  [25.204773  23.265625  20.628296 ]
  [40.881226  35.733215  22.349976 ]
  ...
  [ 0.         4.         7.       ]
  [ 0.         3.         6.       ]
  [ 0.         1.         6.       ]]

 [[30.160156  29.328125  26.       ]
  [36.765625  33.2382

In [30]:
def scale(image, label):
    return image/255, label

In [31]:
train_ds = train_ds.map(scale)

In [32]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.09583333 0.09583333 0.09583333]
****Label:  b'daisy'
****Image:  [0.17254902 0.2        0.17254902]
****Label:  b'daisy'
****Image:  [0.1661152  0.17395833 0.15435049]
****Label:  b'daisy'
****Image:  [0.13382353 0.25539216 0.02552083]
****Label:  b'daisy'
****Image:  [0.47294968 0.39629528 0.1406274 ]
****Label:  b'daisy'
